# Install Dependencies

In [1]:
!pip --default-timeout=100 install pyspark -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


# Read Json

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("COMP5349 A2 Data Loading") \
    .getOrCreate()

data = "Assignment_2_data/test.json"
init_df = spark.read.json(data)

In [3]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import udf

data_df= init_df.select((explode("data").alias('data')))
paragraph_df = data_df.select(explode("data.paragraphs").alias("paragraph"))

# data_df.printSchema()
# paragraph_df.printSchema()
# paragraph_df.show(5)

In [4]:
paragraph_unrolled_df = paragraph_df.select("paragraph.context" , explode("paragraph.qas").alias("qas")) \
                                    .select("context", "qas.id", "qas.question", "qas.is_impossible", explode("qas.answers").alias("answer")) \
                                    .select("context", "id", "question", "is_impossible", "answer.answer_start", "answer.text")

paragraph_unrolled_df.show(5)
# paragraph_unrolled_df.printSchema()

+--------------------+--------------------+--------------------+-------------+------------+--------------------+
|             context|                  id|            question|is_impossible|answer_start|                text|
+--------------------+--------------------+--------------------+-------------+------------+--------------------+
|Exhibit 10.16 SUP...|LohaCompanyltd_20...|Highlight the par...|        false|          14|     SUPPLY CONTRACT|
|Exhibit 10.16 SUP...|LohaCompanyltd_20...|Highlight the par...|        false|         143|         The seller:|
|Exhibit 10.16 SUP...|LohaCompanyltd_20...|Highlight the par...|        false|          49|The buyer/End-Use...|
|Exhibit 10.16 SUP...|LohaCompanyltd_20...|Highlight the par...|        false|       10985|The Contract is v...|
|Exhibit 10.16 SUP...|LohaCompanyltd_20...|Highlight the par...|        false|       10691|It will be govern...|
+--------------------+--------------------+--------------------+-------------+------------+-----

In [99]:
import re
from pyspark.sql.types import ArrayType, StringType, LongType
from pyspark.sql.functions import split

@udf(returnType = ArrayType(StringType()))
def segmentContext(str):
    res = [repr(i)+','+str[i:i+4096] for i in range(0, len(str) - 2048, 2048)]
    return res

# res = segmentContext("1111122222\n\n\n\n\n333334444455")
# print(res)

paragraph_preproc_df = paragraph_unrolled_df.withColumn("sequence", explode(segmentContext("context"))).drop("context") \
                                            .withColumn("index", split("sequence",",").getItem(0)) \
                                            .withColumn("source", split("sequence",",").getItem(1)) \
                                            .drop("sequence")
# paragraph_preproc_df.show(5)
# paragraph_preproc_df.printSchema()

+--------------------+--------------------+-------------+------------+---------------+-----+--------------------+
|                  id|            question|is_impossible|answer_start|           text|index|              source|
+--------------------+--------------------+-------------+------------+---------------+-----+--------------------+
|LohaCompanyltd_20...|Highlight the par...|        false|          14|SUPPLY CONTRACT|    0|Exhibit 10.16 SUP...|
|LohaCompanyltd_20...|Highlight the par...|        false|          14|SUPPLY CONTRACT| 2048|                 F-1|
|LohaCompanyltd_20...|Highlight the par...|        false|          14|SUPPLY CONTRACT| 4096|r the goods in ac...|
|LohaCompanyltd_20...|Highlight the par...|        false|          14|SUPPLY CONTRACT| 6144|alue and date of ...|
|LohaCompanyltd_20...|Highlight the par...|        false|          14|SUPPLY CONTRACT| 8192|e Sellers shall a...|
+--------------------+--------------------+-------------+------------+---------------+--

In [101]:
from pyspark.sql.functions import col
@udf(returnType = LongType())
def getEndPos(startPos, text):
    length = len(text)
    return startPos + length

@udf(returnType = StringType())
def getOverlappedPos(start, end, index):
    if start > int(index) and start < int(index) + 4096:
        if end < int(index) + 4096 :
            return str(start) + ','+ str(end)
        else:
            return str(start) + ','+ str(index+4096)
    if end > int(index) and end < int(index) + 4096:
        if start < int(index):
            return str(index) + ','+ str(end)
    else:
        return "0,0"
    

positive_sample_preproc_df = paragraph_preproc_df.filter("is_impossible==False").drop("is_impossible") \
                                         .withColumn("answer_end", getEndPos("answer_start", "text")) \
                                         .withColumn("overlapped", getOverlappedPos("answer_start", "answer_end", "index")) \
                                         .drop("answer_start").drop("answer_end") \
                                         .withColumn("answer_start", split("overlapped",",").getItem(0).cast("int")) \
                                         .withColumn("answer_end", split("overlapped",",").getItem(1).cast("int")) \
                                         .drop("overlapped").drop("index") \
                                         .filter((col("answer_start") != 0) & (col("answer_end") != 0))
#                                          .select("source", "question", "answer_start", "answer_end")
# positive_sample_preproc_df.printSchema()               
positive_sample_preproc_df.show(5)

Py4JJavaError: An error occurred while calling o2345.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 50.0 failed 1 times, most recent failure: Lost task 0.0 in stage 50.0 (TID 51) (MyMechRevo executor driver): java.net.SocketException: Software caused connection abort: socket write error
	at java.net.SocketOutputStream.socketWrite0(Native Method)
	at java.net.SocketOutputStream.socketWrite(Unknown Source)
	at java.net.SocketOutputStream.write(Unknown Source)
	at java.io.BufferedOutputStream.flushBuffer(Unknown Source)
	at java.io.BufferedOutputStream.write(Unknown Source)
	at java.io.DataOutputStream.write(Unknown Source)
	at java.io.FilterOutputStream.write(Unknown Source)
	at org.apache.spark.api.python.PythonRDD$.write$1(PythonRDD.scala:295)
	at org.apache.spark.api.python.PythonRDD$.$anonfun$writeIteratorToStream$1(PythonRDD.scala:307)
	at org.apache.spark.api.python.PythonRDD$.$anonfun$writeIteratorToStream$1$adapted(Pytho